In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Reading in the data

In [2]:
#read in the data
mid = pd.read_csv("experiment MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())
#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)

#time["Elapsed time, mass 40"].describe()

       mass 2.0_value  mass 4.0_value  mass 5.5_value  mass 12.0_value  \
count    2.349000e+03    2.349000e+03    2.349000e+03     2.349000e+03   
mean     2.620820e-11    8.305234e-13    1.778689e-14     2.530263e-13   
std      6.053045e-12    4.745305e-13    5.342895e-15     5.084379e-14   
min      2.246530e-11    5.100000e-13    6.300000e-15     1.317000e-13   
25%      2.369670e-11    5.876000e-13    1.470000e-14     2.258000e-13   
50%      2.439550e-11    5.983000e-13    1.730000e-14     2.406000e-13   
75%      2.565810e-11    6.234000e-13    2.010000e-14     2.579000e-13   
max      8.010900e-11    3.863000e-12    1.144000e-13     5.491000e-13   

       mass 15.0_value  mass 16.0_value  mass 18.0_value  mass 28.0_value  \
count     2.349000e+03     2.349000e+03     2.349000e+03     2.349000e+03   
mean      5.220085e-14     5.669347e-13     9.225058e-13     1.512949e-12   
std       5.133547e-14     1.516972e-13     3.648962e-13     5.225907e-13   
min       1.250000e-14   

,mass 2.0_time,mass 4.0_time,mass 5.5_time,mass 12.0_time,mass 15.0_time,mass 16.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 40.0_time,...,"Elapsed time, mass 4","Elapsed time, mass 5.5","Elapsed time, mass 12","Elapsed time, mass 15","Elapsed time, mass 16","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 40","Elapsed time, mass 44"
2,2022/12/13 12:30:19.001,2022/12/13 12:30:19.251,2022/12/13 12:30:19.501,2022/12/13 12:30:19.752,2022/12/13 12:30:20.002,2022/12/13 12:30:20.252,2022/12/13 12:30:20.502,2022/12/13 12:30:20.752,2022/12/13 12:30:21.002,2022/12/13 12:30:21.253,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,2022/12/13 12:30:25.326,2022/12/13 12:30:25.576,2022/12/13 12:30:25.826,2022/12/13 12:30:26.077,2022/12/13 12:30:26.327,2022/12/13 12:30:26.577,2022/12/13 12:30:26.827,2022/12/13 12:30:27.077,2022/12/13 12:30:27.328,2022/12/13 12:30:27.578,...,6.325,6.325,6.325,6.325,6.325,6.325,6.325,6.326,6.325,6.325
4,2022/12/13 12:30:31.651,2022/12/13 12:30:31.901,2022/12/13 12:30:32.152,2022/12/13 12:30:32.402,2022/12/13 12:30:32.652,2022/12/13 12:30:32.902,2022/12/13 12:30:33.152,2022/12/13 12:30:33.402,2022/12/13 12:30:33.653,2022/12/13 12:30:33.903,...,12.650,12.651,12.650,12.650,12.650,12.650,12.650,12.651,12.650,12.651
5,2022/12/13 12:30:37.977,2022/12/13 12:30:38.227,2022/12/13 12:30:38.477,2022/12/13 12:30:38.727,2022/12/13 12:30:38.977,2022/12/13 12:30:39.228,2022/12/13 12:30:39.478,2022/12/13 12:30:39.728,2022/12/13 12:30:39.979,2022/12/13 12:30:40.229,...,18.976,18.976,18.975,18.975,18.976,18.976,18.976,18.977,18.976,18.976
6,2022/12/13 12:30:44.302,2022/12/13 12:30:44.552,2022/12/13 12:30:44.802,2022/12/13 12:30:45.052,2022/12/13 12:30:45.302,2022/12/13 12:30:45.553,2022/12/13 12:30:45.803,2022/12/13 12:30:46.053,2022/12/13 12:30:46.304,2022/12/13 12:30:46.554,...,25.301,25.301,25.300,25.300,25.301,25.301,25.301,25.302,25.301,25.301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,2022/12/13 16:37:24.802,2022/12/13 16:37:25.052,2022/12/13 16:37:25.302,2022/12/13 16:37:25.552,2022/12/13 16:37:25.802,2022/12/13 16:37:26.053,2022/12/13 16:37:26.303,2022/12/13 16:37:26.554,2022/12/13 16:37:26.804,2022/12/13 16:37:27.054,...,14825.801,14825.801,14825.800,14825.800,14825.801,14825.801,14825.802,14825.802,14825.801,14825.801
2347,2022/12/13 16:37:31.127,2022/12/13 16:37:31.377,2022/12/13 16:37:31.627,2022/12/13 16:37:31.877,2022/12/13 16:37:32.127,2022/12/13 16:37:32.378,2022/12/13 16:37:32.628,2022/12/13 16:37:32.879,2022/12/13 16:37:33.129,2022/12/13 16:37:33.379,...,14832.126,14832.126,14832.125,14832.125,14832.126,14832.126,14832.127,14832.127,14832.126,14832.126
2348,2022/12/13 16:37:37.452,2022/12/13 16:37:37.702,2022/12/13 16:37:37.952,2022/12/13 16:37:38.202,2022/12/13 16:37:38.452,2022/12/13 16:37:38.703,2022/12/13 16:37:38.953,2022/12/13 16:37:39.204,2022/12/13 16:37:39.454,2022/12/13 16:37:39.704,...,14838.451,14838.451,14838.450,14838.450,14838.451,14838.451,14838.452,14838.452,14838.451,14838.451
2349,2022/12/13 16:37:43.777,2022/12/13 16:37:44.027,2022/12/13 16:37:44.277,2022/12/13 16:37:44.527,2022/12/13 16:37:44.777,2022/12/13 16:37:45.028,2022/12/13 16:37:45.278,2022/12/13 16:37:45.528,2022/12/13 16:37:45.779,2022/12/13 16:37:46.029,...,14844.776,14844.776,14844.775,14844.775,14844.776,14844.776,14844.776,14844.777,14844.776,NaN


In [3]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,13/12/2022 10:16:52,7.0,-0.047109,1.610000e-09,1.700000e-09,4.996000e-09,3.883000e-07,0.79667,264.0,-5.163,4270.802,NaN,NaN
2,NaN,13/12/2022 10:16:59,14.0,-0.049523,1.610000e-09,1.710000e-09,4.996000e-09,3.887000e-07,0.79667,264.0,-5.387,4271.251,NaN,NaN
3,NaN,13/12/2022 10:17:06,21.0,-0.050807,1.620000e-09,1.720000e-09,4.996000e-09,3.887000e-07,0.79667,265.0,-5.163,4271.700,NaN,NaN
4,NaN,13/12/2022 10:17:14,29.0,-0.046664,1.620000e-09,1.730000e-09,4.996000e-09,3.887000e-07,0.79667,264.0,-5.050,4272.261,NaN,NaN
5,NaN,13/12/2022 10:17:21,36.0,-0.046688,1.620000e-09,1.730000e-09,4.996000e-09,3.892000e-07,0.79667,265.0,-5.050,4271.139,NaN,NaN


In [4]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)
hv.drop(index=hv.index[0], 
        axis=0, 
        inplace=True)

hv.describe()



,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,hv_grid,em
count,17990.000000,17990.000000,17990.000000,17990.000000,17990.000000,17990.000000,17990.0,17990.0,17990.000000,17990.000000,17990.000000,17990.000000,17990.000000,17990.000000,17990.000000,17990.000000,17990.000000,1.799000e+04,1.799000e+04
mean,10736.791928,148.642030,72.602601,723.543123,723.567679,723.454823,0.0,0.0,0.010310,-0.170448,0.631303,23.885902,23.903297,23.900375,-0.012947,-0.028290,-0.011917,1.833553e-07,6.376525e-07
std,6218.629470,228.322833,44.641961,447.191660,447.260362,447.348303,0.0,0.0,0.009995,0.211052,0.731377,14.795187,14.808843,14.803129,0.002747,0.001214,0.001854,1.999397e-07,7.258474e-07
min,1.185000,0.000000,0.080000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,-0.593300,-3.616800,-1.335800,-0.948500,-0.585400,-0.018700,-0.031400,-0.016800,0.000000e+00,9.000000e-10
25%,5331.956250,0.060000,0.080000,0.080000,0.120000,0.000000,0.0,0.0,0.000000,-0.407500,0.011000,0.006200,0.002700,0.008700,-0.015500,-0.029400,-0.012700,4.500000e-09,1.100000e-08
50%,10725.534500,0.140000,100.080000,1000.040000,1000.160000,1000.160000,0.0,0.0,0.020000,-0.003900,0.028700,32.656100,32.684800,32.673100,-0.012200,-0.028300,-0.011700,2.110000e-08,2.870000e-08
75%,16126.438250,499.960000,100.080000,1000.060000,1000.180000,1000.180000,0.0,0.0,0.020000,0.020000,1.418400,33.671900,33.698425,33.689800,-0.011100,-0.027300,-0.010800,4.076000e-07,1.419700e-06
max,21520.028000,499.980000,100.160000,1000.080000,1000.200000,1000.180000,0.0,0.0,0.020000,0.525300,3.293600,34.384600,34.409100,34.406300,-0.008000,-0.025000,-0.007700,5.933000e-07,3.616800e-06


### Data processing

In [5]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [22]:
#Filtering background signal

#Find the timestamp where emission parameters were changed
print("Result 1:",hv.loc[hv["Comment"]=="grid repelling"])
print("Result 2:", hv.loc[hv["Comment"]=="grid transparent again. stable conditions"])
x=pd.Timestamp(2022, 12, 13, 14, 44, 38)
y=pd.Timestamp(2022, 12, 13, 15, 12, 30)

#then find the time value closest to it in the MID scan file 
print("\n","Result 3:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])

#take a mass with a signal known to be in the background
bck=np.mean(mid["mass 40.0_value"])

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck


#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][920:975]/h2_sens
print(np.mean(mid["mass 2, corr"][920:975]))

#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][1276:1540]/h2_sens
print(np.mean(mid["mass 2, corr"][1276:1540]))

Result 1:              Date          Time  Elapsed time   VMon1   VMon2    VMon3  \
6123   13/12/2022  12:40:37.740      7275.201   70.44  100.10  1000.04   
6124   13/12/2022  12:40:38.943      7276.404  200.86  100.16  1000.04   
6125   13/12/2022  12:40:40.144      7277.605  291.26  100.12  1000.04   
6126   13/12/2022  12:40:41.359      7278.820  421.34  100.12  1000.04   
6127   13/12/2022  12:40:42.561      7280.022  499.22  100.12  1000.04   
12315  13/12/2022  14:44:33.650     14711.111   40.60  100.08  1000.06   
12316  13/12/2022  14:44:34.842     14712.303  170.62  100.14  1000.06   
12317  13/12/2022  14:44:36.048     14713.509  311.14  100.14  1000.06   
12318  13/12/2022  14:44:37.251     14714.712  441.42  100.12  1000.06   

         VMon4    VMon5  VMon6  VMon7  ...   IMon2    IMon3    IMon4    IMon5  \
6123   1000.16  1000.16    0.0    0.0  ...  1.7841  31.4468  31.3821  31.4624   
6124   1000.16  1000.16    0.0    0.0  ...  0.4295  31.5271  31.4590  31.4800   
6125  

### Plotting

In [7]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))
plt.plot(time['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time['Elapsed time, mass 4'],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time['Elapsed time, mass 16'],mid["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(time['Elapsed time, mass 40'],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID scan 1 ML tech. surface pre-injection, EGA out, dynamic vacuum")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }
plt.text(240, 6.1e-11, 'Base pressure gradient from injection', fontdict = font)
plt.text(2710, 1.7e-11, 'Grid repelling', fontdict = font)
plt.text(5900, 3e-11, 'Grid transparent', fontdict = font)
plt.text(8110, 1.7e-11, 'Grid repelling', fontdict = font)
plt.text(10000, 3e-11, 'Grid transparent', fontdict = font)
plt.yscale('log')

In [8]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution with 1ML tech. surface pre-injection,EGA out, dynamic vacuum")


plt.yscale('log')


In [9]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution with 1ML tech. surface pre-injection,EGA out, dynamic vacuum")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()

In [10]:
#Plotting the temp evolution
plt.figure()
plt.plot((vaclog["Elapsed time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution with 1ML tech. surface pre-injection,EGA out, dynamic vacuum")
plt.yscale('linear')

# Primary desorption calculation method
### Elena's formula for the cryo setup
$$
\eta=\frac{C_{H2}\cdot (\Delta p_1 - \Delta p_2)q_e}{k_B\cdot T \cdot I_e}
$$

Where $\Delta p_i=p(t)_i - p(t=0)_i$ and $i=1,2$
In the case of the 4K setup, we know the exact conductance of gasses. The formula above applies well.

Due to the EGA being fully retracted, it is safe to assume that the desorption takes place at approximately room temperature

In [23]:
#calculation delta_p-s
#TODO - fix negative value
dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = 1.38e-23
T = 293
q_e = 1.6e-19
c_h2 = 37.91/1000
s_h2=42/1000
x_h2=1/((1/c_h2)+(1/s_h2))
print(x_h2)
#Emission current
#ignore the inrush current values
hv["em"].where(hv["em"] <= 1.5e-6, np.nan, inplace=True)

I_em = np.nanmean(hv["em"].nlargest(30))
print(I_em)
#Conductance for CO in m3/s
c_co=10.13/1000
c_co2 = 8.08/1000


1.5054014084507043e-08
0.01992516581153798
1.4993266666666665e-06


In [24]:
#Desorption yield
eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

The desorption yield for H2 [mol/el]: 0.007916464576052377 

